# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#upgrade the sdk version by running below command in jupyter notebook terminal window 
#and restart the kernal before running this command.

#pip install --upgrade azureml-core
#pip install --upgrade azureml-automl-core
#!pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets,notebooks]

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
import azureml.train.automl
from azureml.pipeline.steps import AutoMLStep



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ATJKUGAQ4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-138827
aml-quickstarts-138827
southcentralus
2c48c51c-bd47-40d4-abbe-fb8eabd19c8c


In [4]:
# define an experiment name
experiment_name = 'Azure_Capstone_Automl'
experiment=Experiment(ws, experiment_name)


In [5]:
#check the experiment
experiment

Name,Workspace,Report Page,Docs Page
Azure_Capstone_Automl,quick-starts-ws-138827,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
#start logging
run = experiment.start_logging()

# create the computer cluster

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for the GPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster isn't already there
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview

We have used Heart failure clinical records data set from Kaggle

This dataset contains the medical records of 299 heart failure patients collected at the Faisalabad Institute of Cardiology and at the Allied Hospital in Faisalabad (Punjab, Pakistan), during April–December 2015 [52, 66]. The patients consisted of 105 women and 194 men, and their ages range between 40 and 95 years old (Table 1). All 299 patients had left ventricular systolic dysfunction and had previous heart failures that put them in classes III or IV of New York Heart Association (NYHA) classification of the stages of heart failure.

We are using binary classification to predict likelihood of death based on heart failure.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
#the data is already loaded into the workspace, this will be our training dataset
hfdat=Dataset.get_by_name(ws, name='heartfailurerecords')

In [9]:
#check the data
hfdat.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [10]:
df= hfdat.to_pandas_dataframe()

In [11]:
#create the test dataset
hfdat_x = df.drop('DEATH_EVENT', axis= 1)
hfdat_x.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
# TODO: Put your automl settings here
#Will use accuracy as the main metric
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
#Enable early stopping, the task will be classification
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=hfdat,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion(show_output=True)




Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

{'runId': 'AutoML_836ef666-532b-411f-9d9d-b75ea98a5595',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-02-13T18:27:20.928375Z',
 'endTimeUtc': '2021-02-13T18:54:51.147232Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"Azure_Capstone_Automl","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-138827","workspace_name":"quick-starts-ws-138827","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"f

In [15]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [17]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                                                                                               n_estimators=25,
                                                         

In [18]:
print(best_run)

Run(Experiment: Azure_Capstone_Automl,
Id: AutoML_836ef666-532b-411f-9d9d-b75ea98a5595_44,
Type: azureml.scriptrun,
Status: Completed)


In [89]:
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

{'f1_score_macro': 0.8593071717444964, 'f1_score_weighted': 0.8791701604864774, 'AUC_weighted': 0.916349678599221, 'weighted_accuracy': 0.9021085635281896, 'precision_score_weighted': 0.8951582377505298, 'recall_score_macro': 0.8519592768977781, 'average_precision_score_micro': 0.9262492133816993, 'average_precision_score_macro': 0.907082286948637, 'f1_score_micro': 0.8829885057471264, 'recall_score_weighted': 0.8829885057471264, 'AUC_micro': 0.9247351037125116, 'balanced_accuracy': 0.8519592768977781, 'average_precision_score_weighted': 0.92858465474442, 'AUC_macro': 0.916349678599221, 'recall_score_micro': 0.8829885057471264, 'accuracy': 0.8829885057471264, 'precision_score_micro': 0.8829885057471264, 'log_loss': 0.3540782871382714, 'precision_score_macro': 0.8919502931708813, 'norm_macro_recall': 0.7039185537955561, 'matthews_correlation': 0.7403668222607004, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_e187a261-2384-4de6-83f3-41b181512d73_36/accuracy_table', 'confu

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
description = 'HF_AutoML_BestModel'
model = remote_run.register_model(description = description)


In [20]:
print(model.name, model.id, model.version, model.description, sep ='\t')

AutoML836ef666544	AutoML836ef666544:1	1	HF_AutoML_BestModel


In [21]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-138827', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-138827'), name=AutoML836ef666544, id=AutoML836ef666544:1, version=1, tags={}, properties={})


In [34]:
%%writefile env.yml
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment.get(ws, "AzureML-AutoML")
env.inferencing_stack_version='latest'
env.save_to_directory('./',overwrite-False)



Overwriting env.yml


In [23]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment.get(ws, "AzureML-AutoML")
env.inferencing_stack_version='latest'

In [32]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [33]:
from azureml.core.webservice import AciWebservice 
from azureml.core import Model

dep_cfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights= True, auth_enabled= True) 

webservice = Model.deploy(ws, "capstone-automl1", [model], inference_config, dep_cfg)

webservice.wait_for_deployment(show_output = True) 
print(webservice.state) 


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [26]:
print(webservice.get_logs())
print(webservice.scoring_uri) 
print(webservice.swagger_uri)

2021-02-13T18:58:10,757861900+00:00 - rsyslog/run 
2021-02-13T18:58:10,759022800+00:00 - iot-server/run 
2021-02-13T18:58:10,756791600+00:00 - gunicorn/run 
2021-02-13T18:58:10,812076800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
import json


input_payload = json.dumps({
    'data': hfdat_x[0:2].to_dict(orient= 'records')
    
})



In [28]:
print(input_payload)

{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 7861, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 6}]}


In [29]:
output = webservice.run(input_payload)
print(output)

[1, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
webservice.delete()

In [31]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

